In [ ]:
import os
from os.path import join
import sys
from pathlib import Path

# include app directory into sys.path
parent_dir = Path(os.path.abspath('')).parent
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

import torch as pt
import matplotlib.pyplot as plt
import utils.config as config

plt.rcParams["figure.dpi"] = 180

PRED_HORIZON = 64

VAE_PATH = join(parent_dir, "output", "VAE", "latent_study", config.VAE_model)
OUTPUT_PATH = join(parent_dir, "output", "VAE_FC", "param_study")
MODEL_PATH = join(parent_dir, "output", "VAE_FC", "param_study", f"pred_horizon_{PRED_HORIZON}")

In [ ]:
# load study results
study_results = pt.load(join(MODEL_PATH, "study_results.pt"))
param_combinations = list(study_results.keys())

# find parameter combinations of study and extract test loss
input_width = [int(param_set.split('_')[0]) for param_set in param_combinations]
hidden_size = [int(param_set.split('_')[1]) for param_set in param_combinations]
n_hidden = [int(param_set.split('_')[2]) for param_set in param_combinations]
test_losses = [study_results[param_set][0]["val_loss"].values[-10:].mean() for param_set in param_combinations]

# Sort the indexed losses based on the values (ascending order)
sorted_losses = sorted(list(enumerate(test_losses)), key=lambda x: x[1])
lowest_loss_idx = [index for index, _ in sorted_losses[:5]]

#### Parameter Combination Scatter Plot

In [ ]:
print("The param combinations with the lowest loss: [input_width, hidden_size, n_hidden]")
print([param_combinations[i] for i in lowest_loss_idx]) 

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(input_width, hidden_size, n_hidden, c=test_losses, cmap='viridis', s=100)
ax.set_xlabel("input width")
ax.set_xticks([32])
ax.set_ylabel("hidden layer neurons")
ax.set_zlabel("hidden layers")

cbar = plt.colorbar(ax.scatter(input_width, hidden_size, n_hidden, c=test_losses, cmap='viridis'), pad=0.15)
cbar.set_label('Test Loss')
fig.tight_layout()
fig.savefig(join(OUTPUT_PATH, f"VAE_FC_predhor{PRED_HORIZON}_param_study.png"))


#### Prediction Horizon 1
- medium input sequence length          45
- medium hidden size                    256
- less hidden layers perform better     1

#### Prediction Horizon 2
- medium input sequence length          45
- medium hidden size                    256
- less hidden layers perform better     1

#### Prediction Horizon 6
- medium input sequence length          40
- medium hidden size                    256
- less hidden layers perform better     1

#### Prediction Horizon 25
- large input sequence length           55
- medium hidden size                    256
- medium n hidden layers                3

#### Prediction Horizon 64
- 32
- 128
- 5